# Image Classification: Malaria

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Activation, Dropout
from keras.optimizers import adadelta

### Data Import / Preparation

In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

In [4]:
train_generator = datagen.flow_from_directory(
        'data/malaria-images',
        target_size=(110, 110),
        batch_size=32,
        class_mode='binary')

Found 27558 images belonging to 2 classes.


In [5]:
validation_generator = datagen.flow_from_directory(
    'data/malaria-images', # same directory as training data
    target_size=(110, 110),
    batch_size=32,
    class_mode='binary',
    subset='validation') # set as validation data

Found 5510 images belonging to 2 classes.


## Modeling

The next few cells will demonstrate the use of a Convolutional Neural Network to classify these images.

We'll have 1 model, and adjust some parameters. It's important to note that this model will be trained from scratch, and will use no transfer learning of anykind.

In [14]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(110, 110, 3)))
# add an activation layer with ReLU
model.add(Activation('relu'))
# add Dnother convolutional layer
model.add(Conv2D(32, (3,3)))
# add another activation layer now
model.add(Activation('relu'))
# perform maxPooling
model.add(MaxPooling2D(pool_size=(2,2)))
# finally add the dropout layer
# add the dropout layer now, and drop out 25%
model.add(Dropout(0.25))
# same as before just increase kernal size by 2
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
## our final layer bunch
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(train_generator.num_classes))
model.add(Activation('softmax'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 110, 110, 32)      896       
_________________________________________________________________
activation_4 (Activation)    (None, 110, 110, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 108, 108, 32)      9248      
_________________________________________________________________
activation_5 (Activation)    (None, 108, 108, 32)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 54, 54, 64)        18496     
__________

In [16]:
# compile, fit and evaluate
model.compile(loss='sparse_categorical_crossentropy', optimizer=adadelta(), metrics=['accuracy'])

In [17]:
# Fit the model on the batches generated by datagen.flow().
model.fit_generator(train_generator, epochs=10, steps_per_epoch=200, validation_steps=200, validation_data=validation_generator, use_multiprocessing=True)

Epoch 1/10
200/200 [==============================] - 344s 2s/step - loss: 0.6986 - acc: 0.5713 - val_loss: 0.6257 - val_acc: 0.6104
Epoch 2/10
200/200 [==============================] - 346s 2s/step - loss: 0.4570 - acc: 0.8262 - val_loss: 0.2698 - val_acc: 0.9219
Epoch 3/10
200/200 [==============================] - 383s 2s/step - loss: 0.2943 - acc: 0.9134 - val_loss: 0.4069 - val_acc: 0.9325
Epoch 4/10
200/200 [==============================] - 357s 2s/step - loss: 0.2567 - acc: 0.9217 - val_loss: 0.2207 - val_acc: 0.9330
Epoch 5/10
200/200 [==============================] - 369s 2s/step - loss: 0.2154 - acc: 0.9331 - val_loss: 0.1733 - val_acc: 0.9430
Epoch 6/10
200/200 [==============================] - 345s 2s/step - loss: 0.1708 - acc: 0.9497 - val_loss: 0.1857 - val_acc: 0.9410
Epoch 7/10
200/200 [==============================] - 374s 2s/step - loss: 0.1824 - acc: 0.9469 - val_loss: 0.1771 - val_acc: 0.9389
Epoch 8/10
200/200 [==============================] - 363s 2s/step - 